# Evaluation of frozen/quantized graph

In [1]:
import os
import cv2
import numpy as np

In [2]:
import urllib

In [3]:
import tensorflow as tf

In [4]:
# Use CPU
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

In [5]:
print("TensorFlow Version: %s" % tf.__version__)

TensorFlow Version: 1.15.0


## WordNet ID vs. words

In [6]:
label_file = 'ImageNetLabels.txt'
if not os.path.exists(label_file):
    dl = urllib.request.urlretrieve(
        'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt',
        label_file)
    
imagenet_labels = np.array(open(label_file).read().splitlines())

In [7]:
words_file = 'words.txt'
if not os.path.exists(words_file):
    dl = urllib.request.urlretrieve(
        'http://image-net.org/archive/words.txt',
        words_file)
    
wn_ids = {}
with open(words_file, 'r') as f:
    for line in f:
        elem = line.split(sep='\t')
        elem[1] = elem[1].replace('\n', '').rsplit(sep=',')[0]
        wn_ids.update( {elem[1]: elem[0]} )

In [8]:
ll = list()
for lb in imagenet_labels:
    ll.append( (wn_ids[lb], lb) )

## Load model

In [10]:
input_graph_def = tf.Graph().as_graph_def()
input_graph_def.ParseFromString(tf.gfile.GFile('saved_model.pb', 'rb').read())

26683736

In [11]:
tf.import_graph_def(input_graph_def, name = '')

In [12]:
# Show node names
# for node in input_graph_def.node:
#     print(node.name)

In [13]:
input_node = 'input'
output_node = 'InceptionV1/Logits/Predictions/Softmax'

In [14]:
# Get input placeholders & tensors
images_in = tf.get_default_graph().get_tensor_by_name(input_node+':0')

In [108]:
# get output tensors
logits = tf.get_default_graph().get_tensor_by_name(output_node+':0')

In [16]:
print('Input  tensor shape: %s' % images_in.shape)
print('Output tensor shape: %s' % logits.shape)

Input  tensor shape: (?, 224, 224, 3)
Output tensor shape: (?, 1001)


In [17]:
labels = tf.placeholder(tf.int32, shape=[None, logits.shape[1].value])

## Load image

In [102]:
img_path = os.path.join('_calib', 'n01530575_10101.png')

In [103]:
# Load as RGB image
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [104]:
# Derived from ImageNet data
MEAN_RGB = np.reshape(np.asarray([0.485, 0.456, 0.406]), [1, 1, 3])
STDDEV_RGB = np.reshape(np.asarray([0.229, 0.224, 0.225]), [1, 1, 3])

In [105]:
x_test = img.astype(np.float32)
# Normalization
x_test = np.divide((x_test/255.0 - MEAN_RGB), STDDEV_RGB)
x_test = np.expand_dims(x_test, axis=0)

In [109]:
# Ground truth
y_test = np.zeros(logits.shape[1].value)
y_test = np.expand_dims(y_test, axis=0)

In [110]:
# Get WordNet ID
img_id = os.path.split(img_path)[1].split('_')[0]
keys = [k for k, v in wn_ids.items() if v == img_id]
idx = ll.index((img_id, keys[0]))
y_test[0][idx] = 1
# print(idx)

***

## Inference

In [112]:
# top 5 and top 1 accuracy
in_top5 = tf.nn.in_top_k(predictions=logits, targets=tf.argmax(labels, 1), k=5)
in_top1 = tf.nn.in_top_k(predictions=logits, targets=tf.argmax(labels, 1), k=1)
top5_acc = tf.reduce_mean(tf.cast(in_top5, tf.float32))
top1_acc = tf.reduce_mean(tf.cast(in_top1, tf.float32))

In [113]:
# Create the Computational graph
with tf.Session() as sess:
    sess.run(tf.initializers.global_variables())

    feed_dict = {images_in: x_test, labels: y_test}
    t5_acc, t1_acc, logits = sess.run([top5_acc, top1_acc, logits], feed_dict)

#     print ('Top 1 accuracy with validation set: {:1.4f}'.format(t1_acc))
#     print ('Top 5 accuracy with validation set: {:1.4f}'.format(t5_acc))

Top 1 accuracy with validation set: 1.0000
Top 5 accuracy with validation set: 1.0000


***

## Show result

In [115]:
print ('Top 1 accuracy with validation set: {:1.4f}'.format(t1_acc))
print ('Top 5 accuracy with validation set: {:1.4f}'.format(t5_acc))

Top 1 accuracy with validation set: 1.0000
Top 5 accuracy with validation set: 1.0000


In [114]:
result = np.argmax(logits, axis=1)
# prob = logits[result]
for r in result:
    print('%s: %.4f' % (ll[r][1], logits[0][r]) )

brambling: 0.627223
